In [1]:
import serial
import binascii

In [2]:
PACKAGE_HEAD = [0xFF, 0xFF]
RES_PACKAGE_HEAD = [0xFF, 0xFF]

In [3]:
def assemble_string(data):
    byte_param = []
    data_list = list(data)
    funname_data = list(map(lambda x: ord(x), data_list))
    return funname_data

In [4]:
def crc_calculate(package):
    crc = 0
    for hex_data in package:

        b2 = hex_data.to_bytes(1, byteorder='little')
        crc = binascii.crc_hqx(b2, crc)

    return [(crc >> 8), (crc & 255)]    # 校验位两位

In [5]:
def action_request(action_name):
    #global action_list 
    #if len(action_list) != 0:
    #    action_name = action_list.pop(0)
    #else:
    #    action_name = 'XiaDun'

    # action_name = 'music3'
    # print('REQUEST!')
    pac_data = assemble_string(action_name)
    print('action_name', action_name)
    print('pac_data', pac_data)
    total_len = 1 + 2 + len(pac_data) + 2

    pac_param_len = [len(pac_data) + 1]
    pac_param_type = [0x03]
    pac_param = pac_param_len + pac_param_type + pac_data
    print('pac_param_len', pac_param_len)
    print('pac_param_type', pac_param_type)
    print('pac_data', pac_data)
    print('pac_param', pac_param)

    pac_len = [( total_len >> 8), (total_len & 255)]
    pac_param_count = [0x01]

    pac_data = RES_PACKAGE_HEAD + pac_len + pac_param_count + pac_param
    pac_check_sum = crc_calculate(pac_data)
    print('pac_check_sum', pac_check_sum)
    res = pac_data + pac_check_sum
    print('res', res)
    return res

In [20]:
def get_serial(a):
    s_data = []
    for i in range(len(a)):
        s_data.append(hex(a[i]))
    for i in range(len(s_data)):
        if s_data[i][-2] == 'x':
            print('0'+s_data[i][-2]+'0'+s_data[i][-1], end=', ')
        else:
            print('0x'+s_data[i][-2:], end=', ')

In [21]:
res = action_request('Stand') #在这里修改动作组名称，名称需要和Actionlib.lua里的函数名一样，才能调用成功
get_serial(res) 
#运行完之后，把这个框输出的最后一行0xff开头的十六进制数据复制到robot_action.ipynb中，仿照使用“Stand”和“XiaDun”的方式使用即可

action_name Stand
pac_data [83, 116, 97, 110, 100]
pac_param_len [6]
pac_param_type [3]
pac_data [83, 116, 97, 110, 100]
pac_param [6, 3, 83, 116, 97, 110, 100]
pac_check_sum [152, 79]
res [255, 255, 0, 10, 1, 6, 3, 83, 116, 97, 110, 100, 152, 79]
0xff, 0xff, 0x00, 0x0a, 0x01, 0x06, 0x03, 0x53, 0x74, 0x61, 0x6e, 0x64, 0x98, 0x4f, 